In [86]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.preprocessing import MinMaxScaler

In [87]:
# data preprocess

df = pd.read_csv("BankChurners.csv")
df = df.iloc[:,:-2]

In [88]:
df.set_index('CLIENTNUM', inplace=True)

In [89]:
dfOrigin = df.copy()

In [90]:
df = pd.get_dummies(df)

In [91]:
scaler = MinMaxScaler()
dfNorm = scaler.fit_transform(df)

In [93]:
# Kmeans Clustering
km = MiniBatchKMeans(n_clusters=4)
dfPred = km.fit_predict(dfNorm)

In [94]:
df['grp'] = dfPred
dfOrigin['grp'] = dfPred

In [98]:
df.groupby('grp').mean().iloc[:,20:30]

,Education_Level_Graduate,Education_Level_High School,Education_Level_Post-Graduate,Education_Level_Uneducated,Education_Level_Unknown,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Unknown,Income_Category_$120K +
grp,,,,,,,,,,
0,0.304471,0.193412,0.044706,0.151059,0.163294,0.000000,0.000000,1.000000,0.000000,0.000000
1,0.316424,0.190844,0.051964,0.146922,0.143829,0.124343,0.758119,0.000000,0.117538,0.000000
2,0.313506,0.209750,0.052326,0.140429,0.146691,0.000000,1.000000,0.000000,0.000000,0.158318
3,0.298855,0.203711,0.053691,0.148835,0.149625,0.136597,0.000000,0.717726,0.145677,0.147256


In [99]:
df.columns

Index(['Customer_Age', 'Dependent_count', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Attrition_Flag_Attrited Customer', 'Attrition_Flag_Existing Customer',
       'Gender_F', 'Gender_M', 'Education_Level_College',
       'Education_Level_Doctorate', 'Education_Level_Graduate',
       'Education_Level_High School', 'Education_Level_Post-Graduate',
       'Education_Level_Uneducated', 'Education_Level_Unknown',
       'Marital_Status_Divorced', 'Marital_Status_Married',
       'Marital_Status_Single', 'Marital_Status_Unknown',
       'Income_Category_$120K +', 'Income_Category_$40K - $60K',
       'Income_Category_$60K - $80K', 'Income_Category_$80K - $120K',
       'Income_Category_Less than $40K', 'Income_Category_Unknown',
       'C